# XGBoost is all you need, Maybe...
Hello Kaggle, in this Notebook I will try to push the limits of XGBoost, let see how I will do...
I will take a lot of inspiration from this Book, basically as I read the book I will implement all the tips in this notebook...

**Hands-On Gradient Boosting with XGBoost and scikit-learn: Perform accessible machine learning and extreme gradient boosting with Python**

https://www.oreilly.com/library/view/hands-on-gradient-boosting/9781839218354/

### Notebook Plan
**Objective:**
Build a powerfull XGBoost Model that can provide a good estimation.

**Strategy:**
I think I will follow this strategy:
* Contruct aggregated features, Min, Max. Mad, Var, Sum and Others. I will need to identify the best grouping strategy to create this features. -- Completed
* Contruct lag features -- Working On
* Contruct rolling features -- Working On
* Contruct Expanding features -- Working On
* Other ideas that I'm not sure at this point -- Researching


---

### Data Descriptions
In this competition, you'll classify 60-second sequences of sensor data, indicating whether a subject was in either of two activity states for the duration of the sequence

### Files and Field Descriptions
train.csv - the training set, comprising ~26,000 60-second recordings of thirteen biological sensors for almost one thousand experimental participants
* sequence - a unique id for each sequence
* subject - a unique id for the subject in the experiment
* step - time step of the recording, in one second intervals
* sensor_00 - sensor_12 - the value for each of the thirteen sensors at that time step
* train_labels.csv - the class label for each sequence.
* sequence - the unique id for each sequence.
* state - the state associated to each sequence. This is the target which you are trying to predict.

test.csv - the test set. For each of the ~12,000 sequences, you should predict a value for that sequence's state.

sample_submission.csv - a sample submission file in the correct format.

---

# 0. Installing Libraries


In [ ]:
%%capture
#Install optuna for hyper-param optimization
!pip install optuna

# 1. Loading the Requiered Libraries
Importing the typical set of libraries to create a ML model, I tried to keep this to the minimun possible.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

___

# 2. Setting the Notebook
In this section I will configure some of the default parameters for my notebook execution. for example number of decimals, warning and numbers of rows I will like to load in case the dataset is to massive.

In [ ]:
%%time
# I like to disable my Notebook Warnings.
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%time
# Notebook Configuration...

# Amount of data we want to load into the Model...
DATA_ROWS = None
# Dataframe, the amount of rows and cols to visualize...
NROWS = 100
NCOLS = 15
# Main data location path...
BASE_PATH = '...'

In [ ]:
%%time
# Configure notebook display settings to only use 2 decimal places, tables look nicer.
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', NCOLS) 
pd.set_option('display.max_rows', NROWS)

___

# 3. Loading the Information (CSV) Into A Dataframe
In this section I just import the datasets CSVs using Pandas, not much to see here this is a simple step

In [ ]:
%%time
# Load the CSVs into a pandas dataframe for future data manipulation.
trn_data = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2022/train.csv')
trn_label_data = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2022/train_labels.csv')
tst_data = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2022/test.csv')

sub = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2022/sample_submission.csv')

# 4. Exploring the Information Available
The typical quick exploration to get an idea of the datsets loaded. I ussually like to load the information of the dataset, number of variables, type of variables.
as also visualizing the first few rowns in the dataframe.


## 4.1. Analysing the Trian Dataset
Simple analysis of the train dataset I ussually run
info, head, descrive and number of nulls

In [ ]:
%%time
trn_data.info()

In [ ]:
%%time
trn_data.head(10)

In [ ]:
%%time
trn_data.describe()

In [ ]:
%%time
trn_data.isnull().sum()

---

## 4.2. Analysing the Trian Labels Dataset

In [ ]:
%%time
trn_label_data.info()

In [ ]:
%%time
trn_label_data.head()

In [ ]:
%%time
trn_label_data.describe()

In [ ]:
%%time
trn_label_data.isnull().sum()

---

## 4.3. Analysing the Trian Dataset, Using Groups

In [ ]:
%%time
trn_summary = trn_data[['sequence', 'subject', 'step']].groupby(['sequence', 'subject']).count().reset_index()

In [ ]:
%%time
trn_summary[trn_summary['subject'] == 66].shape

In [ ]:
%%time
summary_by_subject = trn_summary[['sequence', 'subject']].groupby(['subject']).count().reset_index()
summary_by_subject.head()

In [ ]:
%%time
trn_unique_subjects = set(list(trn_data['subject'].unique()))
tst_unique_subjects = set(list(tst_data['subject'].unique()))
overlap_subjets = trn_unique_subjects.intersection(tst_unique_subjects)
print('Repeated Subjects in Test Dataset:', len(overlap_subjets))

---

# 5. Exploratory Data Analysis

## 5.1. Visualization and Others (Under Construction!)

In [ ]:
# ...

# 6. Creating New Model Features

## 6.1. Creating Aggregated Features by Subject and Sequence

In [ ]:
%%time
from scipy.stats import kurtosis

def kurtosis_func(series: pd.Series) -> pd.Series:
    '''
    Calculates the Kurtosis of a Series.
    '''
    return kurtosis(series)


def q01(series: pd.Series) -> pd.Series:
    '''
    Calculates the 1% percentile quantile of a Series using numpy
    '''
    return np.quantile(series, 0.01)


def q05(series: pd.Series) -> pd.Series:
    '''
    Calculates the 5% percentile quantile of a Series using numpy
    '''
return np.quantile(series, 0.05)


def (series: pd.Series) -> pd.Series:
    '''
    Calculates the 95% percentile quantile of a Series using numpy
    '''
    return np.quantile(series, 0.95)


def (series: pd.Series) -> pd.Series:
    '''
    Calculates the 99% percentile quantile of a Series using numpy
    '''
    return np.quantile(series, 0.99)


def aggregated_features(df: pd.Dataframe, aggregation_cols = ['sequence']: list, prefix = '': str) -> pd.Dataframe :
    '''
    Creates an aggregated dataframe based on the aggregation colums requiered, renaming the field to the
    indicated prefix
    
    Args:
        df: A Dataframe containing the information that needs to be aggregated
        aggregation_cols: A list of fields that will be used in the aggregation process
        prefix: A name that will be used as a prefix on the new aggregated fields
    
    Returns:
        df: A Dataframe with all the new aggregated fields
    
    
    '''
    # Aggretion dictionary, for each of variables in the Dataframe
    
    agg_strategy = {'sensor_00': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median', 'skew', kurtosis_func, q01, q05, q95, q99, 'std'],
                    'sensor_01': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median', 'skew', kurtosis_func, q01, q05, q95, q99, 'std'],
                    'sensor_02': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median', 'skew', kurtosis_func, q01, q05, q95, q99, 'std'],
                    'sensor_03': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median', 'skew', kurtosis_func, q01, q05, q95, q99, 'std'],
                    'sensor_04': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median', 'skew', kurtosis_func, q01, q05, q95, q99, 'std'],
                    'sensor_05': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median', 'skew', kurtosis_func, q01, q05, q95, q99, 'std'],
                    'sensor_06': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median', 'skew', kurtosis_func, q01, q05, q95, q99, 'std'],
                    'sensor_07': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median', 'skew', kurtosis_func, q01, q05, q95, q99, 'std'],
                    'sensor_08': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median', 'skew', kurtosis_func, q01, q05, q95, q99, 'std'],
                    'sensor_09': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median', 'skew', kurtosis_func, q01, q05, q95, q99, 'std'],
                    'sensor_10': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median', 'skew', kurtosis_func, q01, q05, q95, q99, 'std'],
                    'sensor_11': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median', 'skew', kurtosis_func, q01, q05, q95, q99, 'std'],
                    'sensor_12': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median', 'skew', kurtosis_func, q01, q05, q95, q99, 'std'],
                   }
    
    # Create a new group with all the aggregation fields
    group = df.groupby(aggregation_cols).aggregate(agg_strategy)
    # Rename the group based on the prefix
    group.columns = ['_'.join(col).strip() for col in group.columns]
    group.columns = [str(prefix) + str(col) for col in group.columns]
    group.reset_index(inplace = True)
    
    # Create a new temp dataframe to merge the results of the aggregation back
    temp = (df.groupby(aggregation_cols).size().reset_index(name = str(prefix) + 'size'))
    group = pd.merge(temp, group, how = 'left', on = aggregation_cols,)
    
    # Return the aggregated fields
    return group

In [ ]:
%%time
trn_merge_data = aggregated_features(trn_data, aggregation_cols = ['sequence', 'subject'])
tst_merge_data = aggregated_features(tst_data, aggregation_cols = ['sequence', 'subject'])

## 6.2. Creating Aggregated Features by Subject

In [ ]:
%%time
trn_subjects_merge_data = aggregated_features(trn_data, aggregation_cols = ['subject'], prefix = 'subject_')
tst_subjects_merge_data = aggregated_features(tst_data, aggregation_cols = ['subject'], prefix = 'subject_')

In [ ]:
%%time
trn_subjects_merge_data.head()

---

## 6.3 Experimenting with Lags (Under Construction!)

In [ ]:
%%time
#trn_data['sensor_00_lag_01'] = trn_data['sensor_00'].shift(1)
#trn_data['sensor_00_lag_10'] = trn_data['sensor_00'].shift(10)

# 7. Merging the Datasets for Training

In [ ]:
%%time
trn_merge_data = trn_merge_data.merge(trn_label_data, how = 'left', on = 'sequence')

In [ ]:
%%time
trn_merge_data = trn_merge_data.merge(trn_subjects_merge_data, how = 'left', on = 'subject')
tst_merge_data = tst_merge_data.merge(tst_subjects_merge_data, how = 'left', on = 'subject')

In [ ]:
%%time
trn_merge_data.head()

In [ ]:
%%time
tst_merge_data.head()

---

# 8. Post Processing the Information for the Model

In [ ]:
%%time
ignore = ['sequence', 'state', 'subject', 'size']
features = [feat for feat in trn_merge_data.columns if feat not in ignore]
target_feature = 'state'

---

# 9. Creating a Simple Train / Test Split Strategy

In [ ]:
%%time
from sklearn.model_selection import train_test_split
test_size_pct = 0.10
X_train, X_valid, y_train, y_valid = train_test_split(trn_merge_data[features], trn_merge_data[target_feature], test_size = test_size_pct, random_state = 42)

---

# 10. Building a Baseline GBT Model

In [ ]:
%%time
from xgboost  import XGBClassifier

In [ ]:
params = {'n_estimators': 4096,
          'max_depth': 7,
          'learning_rate': 0.15,
          'subsample': 0.95,
          'colsample_bytree': 0.60,
          'reg_lambda': 1.50,
          'reg_alpha': 6.10,
          'gamma': 1.40,
          'random_state': 69,
          'objective': 'binary:logistic',
          'tree_method': 'gpu_hist',
         }

In [ ]:
%%time
xgb = XGBClassifier(**params)
xgb.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], eval_metric = ['auc'], early_stopping_rounds = 128, verbose = 50)

In [ ]:
%%time
from sklearn.metrics import roc_auc_score
preds = xgb.predict_proba(X_valid)[:, 1]
score = roc_auc_score(y_valid, preds)
print(score)

---

# 11. Undertanding Model Behavior, Feature Importance

In [ ]:
%%time
def plot_feature_importance(importance, names, model_type, max_features = 10):
    #Create arrays from feature importance and feature names
    feature_importance = np.array(importance)
    feature_names = np.array(names)

    #Create a DataFrame using a Dictionary
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)

    #Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    fi_df = fi_df.head(max_features)

    #Define size of bar plot
    plt.figure(figsize=(8,6))
    
    #Plot Searborn bar chart
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
    #Add chart labels
    plt.title(model_type + 'FEATURE IMPORTANCE')
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')

In [ ]:
%%time
import seaborn as sns
import matplotlib.pyplot as plt
plot_feature_importance(xgb.feature_importances_,X_train.columns,'XG BOOST ', max_features = 20)

---

# 12. Baseline Model Submission File Generation

In [ ]:
%%time
from sklearn.metrics import roc_auc_score
preds = xgb.predict_proba(tst_merge_data[features])[:, 1]

In [ ]:
%%time
sub['state'] = preds
sub.to_csv('my_submission_041222.csv', index = False)

---

## 13. Hyper-Param Optimization

In [ ]:
import optuna

In [ ]:
%%time
X_train, X_valid, y_train, y_valid = train_test_split(trn_merge_data[features], trn_merge_data[target_feature], test_size = test_size_pct, random_state = 42)

def objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 8, 2048)
    max_depth = trial.suggest_int("max_depth", 2, 16)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.2)
    subsample = trial.suggest_float("subsample", 0.5, 1)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.5, 1)
    reg_lambda = trial.suggest_float("reg_lambda", 1, 20)
    reg_alpha = trial.suggest_float("reg_alpha", 0, 20)
    gamma = trial.suggest_float("gamma", 0, 20)
    min_child_weight  = trial.suggest_int("min_child_weight", 0, 128)
    
    clf = XGBClassifier(n_estimators  = n_estimators,
                       learning_rate = learning_rate,
                       max_depth = max_depth,
                       subsample = subsample,
                       colsample_bytree = colsample_bytree,
                       reg_lambda = reg_lambda,
                       reg_alpha = reg_alpha,
                       gamma = gamma,
                       min_child_weight = min_child_weight,
                       random_state  = 69,
                       objective = 'binary:logistic',
                       tree_method = 'gpu_hist',
                      )
    
    clf.fit(X_train, y_train)
    
    valid_pred = clf.predict_proba(X_valid)[:, 1]
    score = roc_auc_score(y_valid, valid_pred)
    
    return score

In [ ]:
%%time
study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials = 100)

In [ ]:
%%time
parameters = study.best_params
parameters

# 13.